# DraCor API basics\n\nIn this short demo we call the DraCor API using the helper in `utils/dracor_client.py` and preview a couple of results.

In [1]:
from utils import dracor_client as dc\ncorpora = dc.corpora()\ncorpora[:5]

SyntaxError: unexpected character after line continuation character (922888146.py, line 1)

In [ ]:
import pandas as pd\ndf = pd.DataFrame(corpora)\ndf[['id','name']].head()

In [ ]:
# Pick the first corpus and list a few plays\nfirst_corpus = corpora[0]['id']\nplays = dc.corpus_plays(first_corpus)\npd.DataFrame(plays)[['id','title']].head()